In [ ]:
import scipy.sparse as sps
import numpy as np
import numba
import jax

In [ ]:
X = sps.rand(81456, 3987, density=0.0000002, format='csr')
V = np.random.random((3987,1))

In [ ]:
%timeit X @ V

In [ ]:
# compare to https://github.com/scipy/scipy/blob/5ae19560e0bb1934ff3d375fd36603fdf4265464/scipy/sparse/sparsetools/csr.h#L1121
def csr_X_v(X_data, X_rowptr, X_col, N, V):
    res = np.zeros((N,1))
    for i in range(N):
        sum = res[i]
        for k in range(X_rowptr[i], X_rowptr[i+1]):
            sum += X_data[k] * V[X_col[k]]
        res[i] = sum
    return res

In [ ]:
np.all(np.equal(csr_X_v(X.data, X.indptr, X.indices, X.shape[0], V), X @ V).flat)

In [ ]:
%timeit csr_X_v(X.data, X.indptr, X.indices, X.shape[0], V)

In [ ]:
csr_X_v_njit = numba.njit(csr_X_v)
csr_X_v_njit(X.data, X.indptr, X.indices, X.shape[0], V)

In [ ]:
%timeit csr_X_v_njit(X.data, X.indptr, X.indices, X.shape[0], V)

In [ ]:
csr_X_v_njitfm = numba.njit(csr_X_v, fastmath=True)
csr_X_v_njitfm(X.data, X.indptr, X.indices, X.shape[0], V)

In [ ]:
%timeit csr_X_v_njitfm(X.data, X.indptr, X.indices, X.shape[0], V)